In [1]:
import owlready2 as owl
# Java should be already recognized
print(owl.JAVA_EXE)


java


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Load the ontology about the goemetrical figures and see if the attached SPIN rules are able recognized during inference.

In [2]:
onto = owl.get_ontology("../Ontologies/SPIN/geoSWRL.rdf").load()
prefixes = """
    PREFIX : <http://www.semanticweb.org/ezzkn/ontologies/2023/1/geoSWRL#>\n
"""

# SPARQL query to get every instance of Square.
rectangle_query = prefixes + """
    SELECT * WHERE {
        ?object rdf:type :Rectangle
    }
"""

square_query = prefixes + """
    SELECT * WHERE {
        ?object rdf:type :Square
    }
"""

with onto:
    world = onto.world
    rectangles = world.sparql(rectangle_query)
    squares = world.sparql(square_query)

    print("Squares: ")
    for square in squares:
        sq = square[0]
        print(f"--- {sq}; name: {sq.name}, height: {sq.height}, width: {sq.width}, area: {sq.area}")

    print("Rectangles:")
    for rectangle in rectangles:
        rect = rectangle[0]
        print(f"--- {rect}; name: {rect.name}, height: {rect.height}, width: {rect.width}, area: {rect.area}")

Squares: 
--- geoSWRL.square1; name: square1, height: [12.0], width: [12.0], area: []
Rectangles:
--- geoSWRL.rectangle1; name: rectangle1, height: [12.0], width: [12.0], area: []
--- geoSWRL.rectangle2; name: rectangle2, height: [13.0], width: [12.0], area: []


Now, I will create the SWRL rules, to calculate the area and also to infer that whenever we have a rectangle whose height and width are equal then the Rectangle is in fact a Square.

In [3]:
rules = [
    """Rectangle(?p) ^ height(?p, ?h) ^ width(?p, ?w) ^ equal(?w, ?h) -> Square(?p)""",
    """Rectangle(?p) ^ height(?p, ?h) ^ width(?p, ?w) ^ multiply(?a, ?w, ?h) -> area(?p, ?a)"""
]

with onto:
    for rule in rules:
        owl.Imp().set_as_rule(rule)

    # Apply rules with a reasoner. --- (disable the debug)
    owl.sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True, debug=0)

    for rule in onto.rules():
        print(str(rule))

with onto:
    world = onto.world
    rectangles = world.sparql(rectangle_query)
    squares = world.sparql(square_query)

    print("Squares: ")
    for square in squares:
        sq = square[0]
        print(f"--- {sq}; name: {sq.name}, height: {sq.height}, width: {sq.width}, area: {sq.area}")

    print("Rectangles:")
    for rectangle in rectangles:
        rect = rectangle[0]
        print(f"--- {rect}; name: {rect.name}, height: {rect.height}, width: {rect.width}, area: {rect.area}")

Rectangle(?p), height(?p, ?h), width(?p, ?w), multiply(?a, ?w, ?h) -> area(?p, ?a)
Rectangle(?p), height(?p, ?h), width(?p, ?w), equal(?w, ?h) -> Square(?p)
Squares: 
--- geoSWRL.rectangle1; name: rectangle1, height: [12.0], width: [12.0], area: [144]
--- geoSWRL.square1; name: square1, height: [12.0], width: [12.0], area: [144]
Rectangles:
--- geoSWRL.rectangle1; name: rectangle1, height: [12.0], width: [12.0], area: [144]
--- geoSWRL.rectangle2; name: rectangle2, height: [13.0], width: [12.0], area: [156]
